In [1]:
import gpytoolbox as gpy # read_mesh, subdivide, remesh_botsch
import polyscope as ps

dir = "../scenes/mitsuba/classroom/"

In [35]:
import numpy as np
import os

def mean_edge_length(V, F):
    E = gpy.edges(F)
    edge_lengths = np.linalg.norm(V[E[:,0],:] - V[E[:,1],:], axis=1)
    aspect = np.max(edge_lengths) / np.min(edge_lengths)
    return np.quantile(edge_lengths, 0.9), aspect

def mean_area(V, F):
    areas = gpy.doublearea(V, F)
    return np.max(areas)

source_dir = os.path.join(dir, "models")
output_dir = os.path.join(dir, "models_refine")

surfs = []

target_edge_length = 0.2
max_aspect = 200
max_unit_area = 100

try:
    os.makedirs(os.path.join(dir, "models_refine"))
except:
    pass

ps.init()

nV = nF = 0

for file in os.listdir(os.fsencode(source_dir)):
    filename = os.fsdecode(file)
    if not(filename.endswith(".obj")):
        continue
    
    V, F = gpy.read_mesh(os.path.join(source_dir, filename))
    # surfs.append(ps.register_surface_mesh(f"{filename}_init", V, F, enabled=True, edge_width=1.0))

    avg_edge_length, aspect = mean_edge_length(V, F)
    can_subdiv = (aspect < max_aspect) or (mean_area(V, F) > max_unit_area * target_edge_length ** 2)
    if can_subdiv:
        subdivs = max(1, int(np.log2(avg_edge_length / target_edge_length)))
        V, F = gpy.subdivide(V, F, iters = subdivs)
    # V, F = gpy.remesh_botsch(V, F, i = 10, h = target_edge_length)
    nV += V.shape[0]
    nF += F.shape[0]
    gpy.write_mesh(os.path.join(output_dir, filename), V, F)

    surfs.append(ps.register_surface_mesh(f"{filename}_remeshed", V, F, enabled=True, edge_width=1.0 if can_subdiv else None))
    # break

ps.show()
for surf in surfs:
    surf.remove()

print(f"Scene statistics: nV = {nV}, nF = {nF}")

Scene statistics: nV = 354682, nF = 530618
